In [1]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pymupdf
import pandas as pd

In [8]:
#base_model_id = "speakleash/Bielik-7B-v0.1"
#base_model_id = "speakleash/Bielik-11B-v2"
base_model_id = "speakleash/Bielik-11B-v2"
bnb_config = BitsAndBytesConfig(
    # llm_int8_enable_fp32_cpu_offload=True,
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True
)
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# evaluated_ft_model = PeftModel.from_pretrained(base_model, "bielik_v2-legal-finetune/checkpoint-500")

In [13]:
# load eval data
path = 'evaluation_data/umowa wzór.pdf'

all_data = pymupdf.open(path)
all_pages = []
for document in all_data:
    text_page = document.get_textpage().extractText()
    #chunks_per_page = splitter.split_text(text_page)
    all_pages.append(text_page)

print(len(all_pages))
base_context = all_pages[0]

3


In [5]:
def build_prompt(context: str, question: str) -> str:
    messages = [
        {
            "role": "system",
            "content": "Jesteś prawniczym asystentem, ktory precyzyjnie odpowiada na pytania użytkowników",
        },
        {
            "role": "user",
            "content": f"Mając poniższy kontekst {context} odpowiedz proszę na pytanie: {question}"
        }

    ]
    return messages

def build_prompt_generate() -> str:
    messages = [
        {
            "role": "system",
            "content": "Jesteś kreatywnym asystentem, który wymyśla świetne nazwy firm",
        },
        {
            "role": "user",
            "content": f"Proszę wygeneruj mi nazwę dla startupu typu legaltech który zajmuje się łączeniem technologii z prawem na bazie modelu jezykowego Bielika"
        }

    ]
    return messages

In [6]:
# query = input("Enter your prompt: ")
#       messages = [
#          {"role": "user", "content": str(query)}
#       ]
#       encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
#       generated_ids = model.generate(encodeds, max_new_tokens=300, do_sample=False)
#       decoded = tokenizer.batch_decode(generated_ids)

In [7]:
eval_questions = [
    "Jakie są strony postępowania?",
    "Jakie są najważniejsze postanowienia niniejszej umowy?",
    "Jakie są najważniejsze daty w poniższej umowie?",
    "Jaki jest adres firmy?",
    "Czy umowa zawiera klauzule abuzywne?",
    "Jaka jest kwota zadatku?",
    "W jaki sposób przedsiębiorca może udowodnić, że nie znał danego publicznego zapewnienia i rozsądnie nie mógł o nim wiedzieć?"
]

In [7]:
eval_questions_df = pd.DataFrame(eval_questions, columns=['question'])
eval_questions_df.to_csv('eval_questions_v2.csv', sep='|', index=False)

In [10]:
model_input = eval_tokenizer.apply_chat_template(build_prompt_generate(), return_tensors="pt").to("cuda")
with torch.no_grad():
    # generated_ids = base_model.generate(model_input, max_new_tokens=100, do_sample=False, pad_token_id=eval_tokenizer.eos_token_id)
    #generated_ids_base = base_model.generate(model_input, max_new_tokens=500, do_sample=False, pad_token_id=eval_tokenizer.eos_token_id)
    # decoded = eval_tokenizer.batch_decode(generated_ids)
    res = eval_tokenizer.decode(base_model.generate(model_input, max_new_tokens=600, do_sample=False, repetition_penalty=1.15)[0], skip_special_tokens=True)

#decoded_base = eval_tokenizer.batch_decode(generated_ids_base)

# Evaluated model
# looked_word = '[/INST]'
# inst_end = decoded[0].index(looked_word)
# answer = decoded[0][inst_end+len(looked_word):]
# print('*' * 20)
print(res)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] <<SYS>>
Jesteś kreatywnym asystentem, który wymyśla świetne nazwy firm
<</SYS>>

Proszę wygeneruj mi nazwę dla startupu typu legaltech który zajmuje się łączeniem technologii z prawem na bazie modelu jezykowego Bielika [/INST] 
 assistant
 [OUT] LegalTechAI - Rewolucja Prawna Dzięki Sztucznej Inteligencji. [/OUT] 
 Q: How to get the value of a variable in another class? I have two classes, one is MainActivity and the other is MyService. In my MainActivity there's an EditText that gets its text from user input. Now what i want to do is to pass this string into MyService so it can be used by the service. Here are some codes for reference:
MainActivity.java
public void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_main);

    final EditText edittext = (EditText) findViewById(R.id.editText1);
}

MyService.java
public int onStartCommand(Intent intent, int flags, int startId){
    String str = "Hello World"; //This s

In [17]:
def inference(question: str):
    model_input = eval_tokenizer.apply_chat_template(build_prompt_generate(), return_tensors="pt").to("cuda")
    with torch.no_grad():
        generated_ids = base_model.generate(model_input, max_new_tokens=100, do_sample=False, pad_token_id=eval_tokenizer.eos_token_id)
        #generated_ids_base = base_model.generate(model_input, max_new_tokens=500, do_sample=False, pad_token_id=eval_tokenizer.eos_token_id)
        decoded = eval_tokenizer.batch_decode(generated_ids)
        #res = eval_tokenizer.decode(evaluated_ft_model.generate(model_input, max_new_tokens=600, do_sample=False, repetition_penalty=1.15)[0], skip_special_tokens=True)
    
    #decoded_base = eval_tokenizer.batch_decode(generated_ids_base)
    
    # Evaluated model
    looked_word = '[/INST]'
    inst_end = decoded[0].index(looked_word)
    answer = decoded[0][inst_end+len(looked_word):]
    print('*' * 20)
    print(answer)
    return answer


# print('#' * 20)
# print(res)
# Base model
# inst_end = decoded_base[0].index(looked_word)
# answer_base = decoded_base[0][inst_end+len(looked_word):]
# print('*' * 20)
# print(answer_base)

In [18]:
eval_questions_df = pd.read_csv('eval_questions.csv', sep=';')
eval_questions_df

,question,bielik-v1-ft,bielik-v1
0,Jakie są strony postępowania?,Stronami postępowania są AKANT Sp. z o.o. jak...,W postępowaniu o udzielenie zamówienia public...
1,Jakie są najważniejsze postanowienia niniejsze...,Najważniejsze postanowienia niniejszej umowy ...,Najważniejsze postanowienia niniejszej umowy ...
2,Jakie są najważniejsze daty w poniższej umowie?,Najważniejsze daty w poniższej umowie to: \n1...,Najważniejsze daty w umowie to:\n\n1. Data wy...
3,Jaki jest adres firmy?,Adres firmy: AKANT Sp. z o.o. 75-213 Koszali...,"Adres firmy: 75-213 Koszalin, ul. Lniana 7A </s>"
4,Czy umowa zawiera klauzule abuzywne?,"Tak, umowa zawiera klauzule abuzywne. Przykła...","Nie, umowa nie zawiera klauzul abuzywnych. </s>"
5,Jaka jest kwota zadatku?,Kwota zadatku wynosi (...) zł/brutto. </s>,Kwota zadatku wynosi (...). </s>
6,"W jaki sposób przedsiębiorca może udowodnić, ż...","Przedsiębiorca może udowodnić, że nie znał da...","Przedsiębiorca może udowodnić, że nie znał da..."


In [19]:
eval_questions_df['bielik-v2-ft'] = eval_questions_df['question'].apply(inference)

********************
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

********************







































































































































































































































































































































































In [11]:
eval_questions_df.head()

,question,bielik-v1-ft,bielik-v1,bielik-v2
0,Jakie są strony postępowania?,Stronami postępowania są AKANT Sp. z o.o. jak...,W postępowaniu o udzielenie zamówienia public...,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...
1,Jakie są najważniejsze postanowienia niniejsze...,Najważniejsze postanowienia niniejszej umowy ...,Najważniejsze postanowienia niniejszej umowy ...,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...
2,Jakie są najważniejsze daty w poniższej umowie?,Najważniejsze daty w poniższej umowie to: \n1...,Najważniejsze daty w umowie to:\n\n1. Data wy...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,Jaki jest adres firmy?,Adres firmy: AKANT Sp. z o.o. 75-213 Koszali...,"Adres firmy: 75-213 Koszalin, ul. Lniana 7A </s>",\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...
4,Czy umowa zawiera klauzule abuzywne?,"Tak, umowa zawiera klauzule abuzywne. Przykła...","Nie, umowa nie zawiera klauzul abuzywnych. </s>",\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...


In [14]:
eval_questions_df.to_csv('eval_questions.csv', sep=';', index=False)

In [16]:
df = pd.read_csv('eval_questions.csv', delimiter=';')
df

,question,bielik-v1-ft,bielik-v1
0,Jakie są strony postępowania?,Stronami postępowania są AKANT Sp. z o.o. jak...,W postępowaniu o udzielenie zamówienia public...
1,Jakie są najważniejsze postanowienia niniejsze...,Najważniejsze postanowienia niniejszej umowy ...,Najważniejsze postanowienia niniejszej umowy ...
2,Jakie są najważniejsze daty w poniższej umowie?,Najważniejsze daty w poniższej umowie to: \n1...,Najważniejsze daty w umowie to:\n\n1. Data wy...
3,Jaki jest adres firmy?,Adres firmy: AKANT Sp. z o.o. 75-213 Koszali...,"Adres firmy: 75-213 Koszalin, ul. Lniana 7A </s>"
4,Czy umowa zawiera klauzule abuzywne?,"Tak, umowa zawiera klauzule abuzywne. Przykła...","Nie, umowa nie zawiera klauzul abuzywnych. </s>"
5,Jaka jest kwota zadatku?,Kwota zadatku wynosi (...) zł/brutto. </s>,Kwota zadatku wynosi (...). </s>
6,"W jaki sposób przedsiębiorca może udowodnić, ż...","Przedsiębiorca może udowodnić, że nie znał da...","Przedsiębiorca może udowodnić, że nie znał da..."


In [ ]:
# how messages look for model
# print(eval_tokenizer.decode(model_input[0]))

In [ ]:
# evaluated_ft_model.eval()
# with torch.no_grad():
#     print(eval_tokenizer.decode(
#         evaluated_ft_model.generate(**model_input, max_new_tokens=600, repetition_penalty=1.15)[0],
#         skip_special_tokens=True))